In [2]:
import re
import json
from PyPDF2 import PdfReader
from collections import defaultdict
from datetime import datetime

In [19]:
def parse_amount(amount_str):
    return float(amount_str.replace(',', '')) if amount_str else 0


In [84]:
def extract_transaction_data(text):
    transactions_by_date = defaultdict(lambda: {"transactions": [], "totals": {"ӨДРИЙН ОРЛОГО": 0, "ӨДРИЙН ЗАРЛАГА": 0}})
    lines = text.splitlines()
    current_date = ""
    printed_date = ""
    account_number = ""
    customer_name = ""
    coverage_period = ""
    for line in lines:
        try:
            printed_date_match = re.search(r'Хэвлэсэн огноо\s*:\s*(\d{4}\.\d{2}\.\d{2})', line)
            if printed_date_match:
                printed_date = printed_date_match.group(1)
            customer_account_match = re.search(r'Харилцагч\s*:\s*([^\(]+)', line)
            if customer_account_match:
                customer_name = customer_account_match.group(1).strip()
            customer_account_number = re.search(r'Дансны дугаар\s*:\s*(\d{9,})', line)
            if customer_account_number:
                account_number = int(customer_account_number.group(1))
            coverage_period_match = re.search(r'(\d{4}\.\d{2}\.\d{2})\s*-\s*(\d{4}\.\d{2}\.\d{2})', line)
            if coverage_period_match:
                coverage_period = coverage_period_match.group(0)
            date_match = re.match(r'(\d{4}/\d{2}/\d{2})', line)
            if date_match:
                print(date_match)
                current_date = date_match.group(0)
            else:
                transaction_match =re.search(r'''
                                    (\d+\s+-\s+\d+)\s+  # Teller
                                    (\d+(?:,\d{3})*(?:\.\d{2})?)?  # Income (optional)
                                    \s*
                                    (\d+(?:,\d{3})*(?:\.\d{2})?)?  # Expense (optional)
                                    \s+
                                    ([\d.]+)\s+  # Exchange rate
                                    (\d+)?\s+  # Related account (optional)
                                    ([\d,]+\.\d{2})\s+  # Balance
                                    (.+)  # Transaction details
                                ''', line , re.VERBOSE | re.MULTILINE)
             
                if transaction_match:
                    amount = float(transaction_match.group(2).replace(',', '').replace(' ', ''))
                    typo_ = transaction_match.group(3)
                    pattern = fr'(\s+){typo_}'
                    matches = re.findall(pattern, transaction_match)
                    for match in matches:
                        spaces_count = len(match)
                    if spaces_count == 3:
                        type_ = 'ОРЛОГО'
                    else:
                        type_ = 'ЗАРЛАГА'
                    details = transaction_match.group(7).strip()
                    transactions_by_date[current_date]["transactions"].append({
                        "amount": amount,
                        "type": type_,
                        "details": details
                    })
                    if type_ == "ОРЛОГО":
                        transactions_by_date[current_date]["totals"]["ӨДРИЙН ОРЛОГО"] += amount
                    elif type_ == "ЗАРЛАГА":
                        transactions_by_date[current_date]["totals"]["ӨДРИЙН ЗАРЛАГА"] += amount
                daily_totals_match = re.search(r'([\d,]+\.\d{2})\s+(ӨДРИЙН ОРЛОГО|ӨДРИЙН ЗАРЛАГА|ӨДРИЙН ҮЛДЭГДЭЛ)', line)
                if daily_totals_match:
                    amount = float(daily_totals_match.group(1).replace(',', '').replace(' ', ''))
                    total_type = daily_totals_match.group(2)
                    transactions_by_date[current_date]["totals"][total_type] = amount
        except Exception as e:
            print(f"Error processing line: {line}\nError: {e}")
    return {
        "Хэвлэсэн огноо": printed_date,
        "Данс": account_number,
        "Харилцагчийн нэр": customer_name,
        "Хамрах хугацаа": coverage_period,
        "transactions_by_date": transactions_by_date
    }
def pdf_to_json(pdf_path, output_json_path):
    try:
        reader = PdfReader(pdf_path)
        full_text = ""
        for page in reader.pages:
            full_text += page.extract_text()
        final_output = extract_transaction_data(full_text)
        with open(output_json_path, 'w', encoding='utf-8') as json_file:
            json.dump(final_output, json_file, ensure_ascii=False, indent=4)
    except Exception as e:
        print(f"Error processing PDF: {e}")
pdf_path = "519ACFDC320C5640.pdf"
output_json_path = "12.json"
pdf_to_json(pdf_path, output_json_path)

<re.Match object; span=(0, 10), match='2024/07/05'>
Error processing line: 06:14:59 PM  490 - 50 484188   1.00 453132756  484,188.00  EB -SALA RY ГрэйпСити 
Error: 'callable_iterator' object is not subscriptable
<re.Match object; span=(0, 10), match='2024/07/06'>
Error processing line: 05:33:21 PM  490 - 50  50000  1.00 5876293731  434,188.00  EB -play 
Error: 'callable_iterator' object is not subscriptable
<re.Match object; span=(0, 10), match='2024/07/06'>
Error processing line: 05:33:21 PM  490 - 50  200 1.00   433,988.00  EB-БАНК ХООРОНД 
Error: 'callable_iterator' object is not subscriptable
<re.Match object; span=(0, 10), match='2024/07/06'>
Error processing line: 05:33:21 PM  490 - 50  100 1.00   433,888.00  EB -И-БАНК 
Error: 'callable_iterator' object is not subscriptable
<re.Match object; span=(0, 10), match='2024/07/07'>
Error processing line: 03:13:08 PM  490 - 50  2500  1.00 429002613  390,788.00  EB -qpay 200000448038897 
Error: 'callable_iterator' object is not subscript

In [51]:
reader = PdfReader(pdf_path)
full_text = ""
for page in reader.pages:
    full_text += page.extract_text()
bruh = re.search(r'\d{4}/\d{2}/\d{2}', full_text)
full_text

' \n                          Депозит дансны хуулга  \n  Хэвлэсэн огноо : 2024.08.08  \n \n \nХарилцагч : БОЛДБАЯР ТҮМЭНЖАРГАЛ   (90499166218 )                                        \nДансны дугаар:  499492549  Эхний үлдэ гдэл : .00 Битүүмж :  \nХамрах хугацаа : 2024.07.01 - 2024.08.08  \n Эцсийн үлдэгдэл : 233,569.00  Боломжит үлдэгдэл : 233,569.00  \n \n \n \nОгноо  Теллер  Орлого  Зарлага  Ханш  Харьцсан \nданс  Үлдэгдэл  Гүйлгээний утга  \n2024/07/05 \n06:14:59 PM  490 - 50 484188   1.00 453132756  484,188.00  EB -SALA RY ГрэйпСити \nцалин  \n2024/07/06 \n05:33:21 PM  490 - 50  50000  1.00 5876293731  434,188.00  EB -play \n2024/07/06 \n05:33:21 PM  490 - 50  200 1.00   433,988.00  EB-БАНК ХООРОНД \nШИЛЖҮҮЛГИЙН \nШИМТГЭЛ 3.7.2, 3.7.4  \n2024/07/06 \n05:33:21 PM  490 - 50  100 1.00   433,888.00  EB -И-БАНК \nГҮЙЛГЭЭНИЙ \nШИМТГЭЛ  \n2024/07/07 \n03:13:08 PM  490 - 50  2500  1.00 429002613  390,788.00  EB -qpay 200000448038897 \nMBA_PAYMENT17203362\n7621774493 1 PREPAID  \n2024/07/07

In [26]:
transaction_match =re.search(r'''
                                (\d{4}/\d{2}/\d{2})\s+  # Date
                                (\d{2}:\d{2}:\d{2}\s+(?:AM|PM))\s+  # Time
                                (\d+\s+-\s+\d+)\s+  # Teller
                                (\d+(?:,\d{3})*(?:\.\d{2})?)?  # Income (optional)
                                \s*
                                (\d+(?:,\d{3})*(?:\.\d{2})?)?  # Expense (optional)
                                \s+
                                ([\d.]+)\s+  # Exchange rate
                                (\d+)?\s+  # Related account (optional)
                                ([\d,]+\.\d{2})\s+  # Balance
                                (.+)  # Transaction details
                            ''', full_text, re.VERBOSE | re.MULTILINE)
transaction_match

<re.Match object; span=(434, 527), match='2024/07/05 \n06:14:59 PM  490 - 50 484188   1.00 >

In [106]:
import re
from datetime import datetime
import json
transaction_pattern = re.compile(r'''
    (\d{4}/\d{2}/\d{2})\s+  # Date
    (\d{2}:\d{2}:\d{2}\s+(?:AM|PM))\s+  # Time
    (\d+\s+-\s+\d+)\s+  # Teller
    (\d+(?:,\d{3})*(?:\.\d{2})?)?  # Income (optional)
    \s*
    (\d+(?:,\d{3})*(?:\.\d{2})?)?  # Expense (optional)
    \s+
    ([\d.]+)\s+  # Exchange rate
    (\d+)?\s+  # Related account (optional)
    ([\d,]+\.\d{2})\s+  # Balance
    (.+)  # Transaction details
''', re.VERBOSE | re.MULTILINE)

def parse_amount(amount_str):
    return float(amount_str.replace(',', '')) if amount_str else 0

def extract_transactions(text):
    transactions_by_date = {}
    
    for match in transaction_pattern.finditer(text):
        date = datetime.strptime(match.group(1), '%Y/%m/%d').strftime('%Y-%m-%d')
        income = parse_amount(match.group(4))
        expense = parse_amount(match.group(5))
        details = match.group(9).strip()

        transaction = {
            "amount": income if income > 0 else -expense,
            "type": "ОРЛОГО" if income > 0 else "ЗАРЛАГА",
            "details": details,
        }

        if date not in transactions_by_date:
            transactions_by_date[date] = {"transactions": []}
        transactions_by_date[date]["transactions"].append(transaction)

    # Extract header information
    header_pattern = re.compile(r'''
        Харилцагчийн\s+нэр:\s+(.+?)\n
        .*?
        Дансны\s+дугаар:\s+(\d+)\n
        .*?
        Эхлэх\s+огноо:\s+(\d{4}-\d{2}-\d{2})\s+Дуусах\s+огноо:\s+(\d{4}-\d{2}-\d{2})\s+Одоогийн\s+огноо:\s+(\d{4}/\d{2}/\d{2})
    ''', re.DOTALL | re.VERBOSE)

    header_match = header_pattern.search(text)
    if header_match:
        header = {
            "Харилцагчийн нэр": header_match.group(1),
            "Данс": header_match.group(2),
            "Хэвлэсэн огноо": header_match.group(5),
            "Хамрах хугацаа": f"{header_match.group(3)} - {header_match.group(4)}"
        }
    else:
        header = {}

    return {**header, "transactions_by_date": transactions_by_date}
result = extract_transactions(full_text)
print(json.dumps(result, indent=2, ensure_ascii=False))

{
  "transactions_by_date": {
    "2024-07-05": {
      "transactions": [
        {
          "amount": 484188.0,
          "type": "ОРЛОГО",
          "details": "EB -SALA RY ГрэйпСити"
        }
      ]
    },
    "2024-07-06": {
      "transactions": [
        {
          "amount": 50000.0,
          "type": "ОРЛОГО",
          "details": "EB -play"
        },
        {
          "amount": 200.0,
          "type": "ОРЛОГО",
          "details": "EB-БАНК ХООРОНД"
        },
        {
          "amount": 100.0,
          "type": "ОРЛОГО",
          "details": "EB -И-БАНК"
        }
      ]
    },
    "2024-07-07": {
      "transactions": [
        {
          "amount": 2500.0,
          "type": "ОРЛОГО",
          "details": "EB -qpay 200000448038897"
        },
        {
          "amount": 100.0,
          "type": "ОРЛОГО",
          "details": "И-БАНК ГҮЙЛГЭЭНИЙ"
        },
        {
          "amount": 10000.0,
          "type": "ОРЛОГО",
          "details": "EB -j"
        },
  